In [1]:
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
from selenium import webdriver
from bs4 import BeautifulSoup
import requests 
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import cufflinks as cf
import plotly.graph_objects as go 
import plotly as py
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

# Функции

Функция, которая в случае необходимости убирает лишние пробелы, а также меняет запятую на точку, чтобы можно было str превратить во float  для дальнейшей работы. 

In [2]:
def string_in_float(stroka: str):
    stroka = stroka.replace(',', '.')
    stroka = stroka.replace(' ', '')
    stroka = float(stroka)

    return stroka

Функция для того, чтобы распарсить валютную пару доллара/рубль, так как там на каждую дату новая страница и новая URL'ка. Также привожу в соответствующий вид переменную d, чтобы сформировать корректную ссылку.

In [3]:
def parsing_valut(d: str):

    d = str(d).split('-')
    d = '.'.join(d[::-1])

    url = 'https://cbr.ru/currency_base/daily/?UniDbQuery.Posted=True&UniDbQuery.To={}'.format(d)

    response = requests.get(url)
    tree = BeautifulSoup(response.content, 'html.parser')
    cur_exch = tree.find_all('tr')

    list_valut = []

    for i in cur_exch:
        i = i.get_text().strip().split('\n')
        if 'USD' in i:
            list_valut.append({'Дата': d, 'Курс доллара к рублю': i[-1]})
        continue
    return list_valut

Следующую функцию сделал для преобразования элементов полученных из парсинга ключевой ставки.

In [4]:
def transformation_bs4el(row: str):
    
    row = row.get_text().strip().split('\n')

    if len(row) == 3:
        a = row[0].split('\xa0') 
        if a[1] == 'мая':
            a = ' '.join(a[:3]) # я без понятия, что за прикол у ЦБ, но почему то именно в мае у них появляется дополнительный разделитель 'xa0' между месяцем и годом, потому пришлось вписать еще один if
            return [a, row[1]]
        a = ' '.join(a[:2])
        return [a, row[1]]
            
    return row

Функция для преобразования даты из вида "число название месяца (в родительном падеже) год" в формат через 'число.месяц.год 'в численном представлении.

In [5]:
def new_date(nd):
    months_dict = {'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04', 'мая': '05','июня': '06','июля': '07', 'августа': '08', 'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'}
    nd = nd.split()
    nd[1] = months_dict[nd[1]]
    nd = '.'.join(nd)
    return nd

Следующие функции создают массив, в котором будут содержаться даты.

In [6]:
def calendar_kvartal(start: datetime, end: datetime):
    date_range = []

    while start <= end:
        date_range.append(start.date())
        start += relativedelta(months=3) # запрос в GPT: 'Как к дате прибавлять именно месяцы, а не дни (При 'вытягивании ключа' использовал timedelta)?'
    return date_range

In [7]:
def calendar_po_dnyam(start: datetime, end: datetime):
    date_range = []

    while start <= end:
        date_range.append(start.date())
        start += relativedelta(days=1) 
    return date_range

# Парсинг ключевой ставки и ставки рефинансирования


Небольшая заметка по данным

В период с 2013 по 2015 годы помимо КЛЮЧЕВОЙ ставки существовала также ставка РЕФИНАНСИРОВАНИЯ, которая появилась в РФ после распада СССР. Разница между ними в финансовом плане заключается только в сроках кредитования коммерческих банках. По ставке рефинансирования выдается годичный кредит, а по "ключу" - краткосрочный. Исходя из этого, я решил взять данные по ключевой ставке в период совместного существования двух этих ставок, так как среди банков наиболее популярной операцией являются - сделки РЕПО овернайт. Среди них нет большого количества желающих взять годичный кредит, зачастую, для них важно перенести позицию на следующий день (возможно, и не раз), но годичный кредит уже не дает такой маневрености.

С 1-го января 2016 года ставка рефинансирования стала равняться ключевой ставке, потому считаю разумным использовать значение ставки рефинансирования в период с  сентября 1997 года по сентябрь 2013 года, раз в конечном счете сам ЦБ к этому пришёл. Также в дальнейшем будут использовать именно термин "ключевой ставки"

Достаем "ключ"

In [8]:
url = 'https://www.cbr.ru/hd_base/keyrate/?UniDbQuery.Posted=True&UniDbQuery.From=17.09.2013&UniDbQuery.To=07.05.2024'
response = requests.get(url)
response

<Response [200]>

In [9]:
tree = BeautifulSoup(response.content, 'html.parser')

In [10]:
key_rate = tree.find_all('tr')

Я вытянул все строки из таблицы, включая самую первую - название столбцов, именно поэтому я начинаю извлекать данные только со второго элемента списка.

In [11]:
list_key_rate = []

for i in key_rate[1:]:
    list_key_rate.append(transformation_bs4el(i))

In [12]:
df_key_rate = pd.DataFrame(list_key_rate, columns=['Дата', 'Ключевая ставка, в %'])

In [13]:
df_key_rate['Дата'] = pd.to_datetime(df_key_rate['Дата'], format='%d.%m.%Y')
df_key_rate['Ключевая ставка, в %'] = df_key_rate['Ключевая ставка, в %'].apply(string_in_float)

In [14]:
df_key_rate.head()

,Дата,"Ключевая ставка, в %"
0,2024-05-07,16.0
1,2024-05-06,16.0
2,2024-05-03,16.0
3,2024-05-02,16.0
4,2024-04-30,16.0


# Ставка рефинансирования

In [15]:
url = 'https://cbr.ru/statistics/idkp_br/refinancing_rates1/'
response = requests.get(url)
response

<Response [200]>

In [16]:
tree = BeautifulSoup(response.content, 'html.parser')

In [17]:
refin_rate = tree.find_all('tr')

Здесь начинаю вытягивать с 3-его элемента, т.к. во 2-ой строке пишется о "привязке" ставки рефинансирования к "ключу".

In [18]:
list_refin_rate = []

for i in refin_rate[2:]:
    list_refin_rate.append(transformation_bs4el(i))


In [19]:
df_refin_rate = pd.DataFrame(list_refin_rate, columns=['Дата', 'Ключевая ставка, в %'])

In [20]:
df_refin_rate['Дата'] = df_refin_rate['Дата'].apply(new_date)
df_refin_rate['Дата'] = pd.to_datetime(df_refin_rate['Дата'], format='%d.%m.%Y')
df_refin_rate['Ключевая ставка, в %'] = df_refin_rate['Ключевая ставка, в %'].apply(string_in_float)

In [21]:
df_refin_rate.head()

,Дата,"Ключевая ставка, в %"
0,2012-09-14,8.25
1,2011-12-26,8.00
2,2011-05-03,8.25
3,2011-02-28,8.00
4,2010-06-01,7.75


In [22]:
df_key_rate = pd.concat([df_key_rate, df_refin_rate], ignore_index=True)

In [23]:
df_key_rate

,Дата,"Ключевая ставка, в %"
0,2024-05-07,16.0
1,2024-05-06,16.0
2,2024-05-03,16.0
3,2024-05-02,16.0
4,2024-04-30,16.0
...,...,...
2737,1993-06-02,110.0
2738,1993-03-30,100.0
2739,1992-05-23,80.0
2740,1992-04-10,50.0


Конец

In [24]:
df_key_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2742 entries, 0 to 2741
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Дата                  2742 non-null   datetime64[ns]
 1   Ключевая ставка, в %  2742 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 43.0 KB


# Начало парсинга курса доллара отношению к рублю

Создаю массив состоящий из дат, для того чтобы в последующем подставлять их в ссылку для перехода на страницу, где указан курс доллара на данную дату.

In [25]:
start = datetime(1997, 9, 22)
end = datetime(2023, 12, 31)
date_range = []

while start <= end:
    date_range.append(start.date())
    start += relativedelta(days=1)

Собираю курс доллара с этой даты, т.к. именно в этот день был начат расчет индекса МосБиржи.

Запрос в GPT

In [26]:
from concurrent.futures import ThreadPoolExecutor

list_valut = []

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(parsing_valut, d) for d in date_range]

    for future in tqdm(futures, total=len(date_range)):
        list_valut.extend(future.result())

100%|██████████| 9597/9597 [05:45<00:00, 27.75it/s]


In [27]:
df_valut = pd.DataFrame(list_valut)

In [28]:
df_valut['Дата'] = pd.to_datetime(df_valut['Дата'], format='%d.%m.%Y')
df_valut['Курс доллара к рублю'] = df_valut['Курс доллара к рублю'].apply(string_in_float)

In [29]:
df_valut.head()

,Дата,Курс доллара к рублю
0,1997-09-22,5853.5
1,1997-09-23,5855.0
2,1997-09-24,5856.5
3,1997-09-25,5858.5
4,1997-09-26,5860.0


Конец

# Подгрузка данных из файлов

# IMOEX

In [30]:
df_imoex = pd.read_csv('C:/Users/seymu/Desktop/Проект АнДан/IMOEX.csv', encoding='cp1251', delimiter=';', skiprows=2)

In [31]:
df_imoex = df_imoex[['TRADEDATE', 'CLOSE']]
df_imoex.columns = ['Дата', 'Цена закрытия']

In [32]:
df_imoex['Дата'] = pd.to_datetime(df_imoex['Дата'], format='%d.%m.%Y')
df_imoex['Цена закрытия'] = df_imoex['Цена закрытия'].apply(string_in_float) 

In [33]:
df_imoex.head()

,Дата,Цена закрытия
0,1997-09-22,100.00
1,1997-09-23,100.67
2,1997-09-24,99.94
3,1997-09-25,99.46
4,1997-09-26,98.87


# Brent

Так как Россия по большей части живет за счет экспорта, то считаю необходимым учитывать стоимость нефти в качестве важного макроэкономического показателя, от которого зависит состояние экономики РФ. В таблице приведены цены на нефть марки "Brent",  Россия экспортирует нефть марки "Urals", и в отношении нее существует устойчивый дисконт к нефти марки "Brent". Поэтому 
P.S. да, в нынешнее время существует "потолок цен" на нефть из России, но в конечном счете многие журналисткие расследования крупных, релевантных источкников показывают, что "потолок" не работает.

In [34]:
df_brent = pd.read_excel('C:/Users/seymu/Desktop/Проект АнДан/Brent.xlsx')

In [35]:
df_brent.columns = ['Дата', 'Цена, в $/барр']

In [36]:
df_brent.head()

,Дата,"Цена, в $/барр"
0,2024-05-03,82.9600
1,2024-05-02,83.6700
2,2024-05-01,83.5627
3,2024-04-30,85.9564
4,2024-04-29,88.3471


# ВВП

ВВП есть в двух таблицах с 1995 по 2011 гг. и с 2011 по 2023 гг.

In [37]:
df_gdp_11 = pd.read_excel('C:/Users/seymu/Desktop/Проект АнДан/GDP.xlsx', sheet_name='1', skiprows=2, nrows=3, usecols=[i for i in range(10, 64)])
df_gdp_23 = pd.read_excel('C:/Users/seymu/Desktop/Проект АнДан/GDP.xlsx', sheet_name='2', skiprows=2, nrows=3, usecols=[i for i in range(52)])

In [38]:
df_gdp_11.columns = range(df_gdp_11.shape[1])
df_gdp_11 = df_gdp_11.T
df_gdp_11.columns = ['Дата', 'ВВП, в млрд. ₽']

In [39]:
df_gdp_23.columns = range(df_gdp_23.shape[1])
df_gdp_23 = df_gdp_23.T
df_gdp_23.columns = ['Дата', 'ВВП, в млрд. ₽']

# Преобразовать в функцию

In [40]:
date_range_11 = calendar_kvartal(datetime(1997, 9, 30), datetime(2010, 12, 31))
date_range_23 = calendar_kvartal(datetime(2011, 3, 31), datetime(2023, 12, 31))

# ЕЩЕ надо умножить на 1000 где то в районе 97ого года

In [41]:
df_gdp_11['Дата'] = date_range_11
df_gdp_11.tail()

,Дата,"ВВП, в млрд. ₽"
49,2009-12-30,10816.423016
50,2010-03-30,9995.758259
51,2010-06-30,10977.035261
52,2010-09-30,12086.463959
53,2010-12-30,13249.283711


In [42]:
df_gdp_23['Дата'] = date_range_23
df_gdp_23.tail()

,Дата,"ВВП, в млрд. ₽"
47,2022-12-30,42532.309641
48,2023-03-30,36176.000693
49,2023-06-30,40070.181189
50,2023-09-30,45087.686145
51,2023-12-30,50814.443887


In [43]:
df_gdp = pd.concat([df_gdp_11, df_gdp_23], ignore_index=True)

In [44]:
df_gdp['Дата'] = pd.to_datetime(df_gdp['Дата'])
df_gdp['ВВП, в млрд. ₽'] = df_gdp['ВВП, в млрд. ₽'].astype(float)

In [45]:
df_gdp

,Дата,"ВВП, в млрд. ₽"
0,1997-09-30,634.159200
1,1997-12-30,640.909200
2,1998-03-30,550.865500
3,1998-06-30,602.453000
4,1998-09-30,675.457100
...,...,...
101,2022-12-30,42532.309641
102,2023-03-30,36176.000693
103,2023-06-30,40070.181189
104,2023-09-30,45087.686145


# ИПЦ

In [46]:
df_ipc = pd.read_excel('C:/Users/seymu/Desktop/Проект АнДан/CPI.xlsx', sheet_name='01', skiprows=[0, 1, 2, 4], nrows=12, usecols=[i for i in range(7, 34)])

# Запрос в GPT 
df = pd.DataFrame({'1997': [107, 108], '1998': [109, 110]})
df
df.index = ['Январь', 'Февраль']
сделай то же самое для этого dataframe'а


df = pd.DataFrame({'Январь': [107, 108], 'Февраль': [109, 110]})

df.index = [1997, 1998]

у меня есть такой DataFrame 
я хочу получить в конечном итоге следующую таблицу 
df_2 = pd.DataFrame({'Дата':['30.01.1997', '28.02.1997', '30.01.1998', '28.02.1998'], 'Значение': ['107',  '109', '108', '110']})
как это реализовать внутри питона не вручную


# Переписать под дататайм

In [47]:
months = ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
df_ipc.index = months

years = [str(year) for year in range(1997, 2024)]
df_ipc.columns = years

df_ipc_melted = df_ipc.reset_index().melt(id_vars=['index'], var_name='Год', value_name='Инфляция, в %')

df_ipc_melted.rename(columns={'index': 'Месяц'}, inplace=True)

days = {'Январь': '31', 'Февраль': '28', 'Март': '31', 'Апрель': '30', 'Май': '31',
    'Июнь': '30', 'Июль': '31', 'Август': '31', 'Сентябрь': '30',
    'Октябрь': '31', 'Ноябрь': '30', 'Декабрь': '31'}

months_num = {'Январь': '01', 'Февраль': '02', 'Март': '03', 'Апрель': '04', 'Май': '05',
    'Июнь': '06', 'Июль': '07', 'Август': '08', 'Сентябрь': '09',
    'Октябрь': '10', 'Ноябрь': '11', 'Декабрь': '12'}

df_ipc_melted['Дата'] = df_ipc_melted.apply(lambda row: f"{days[row['Месяц']]}.{months_num[row['Месяц']]}.{row['Год']}", axis=1)

df_ipc = df_ipc_melted[['Дата', 'Инфляция, в %']]

df_ipc

,Дата,"Инфляция, в %"
0,31.01.1997,102.34
1,28.02.1997,101.54
2,31.03.1997,101.43
3,30.04.1997,100.96
4,31.05.1997,100.94
...,...,...
319,31.08.2023,100.28
320,30.09.2023,100.87
321,31.10.2023,100.83
322,30.11.2023,101.11


In [48]:
df_ipc['Дата'] = pd.to_datetime(df_ipc['Дата'], format='%d.%m.%Y')

# Платежный баланс

In [49]:
col = [0] + list(range(15, 121))
df_bp = pd.read_excel('C:/Users/seymu/Desktop/Проект АнДан/BP.xlsx', sheet_name='Кварталы', skiprows=4, usecols=col)

In [50]:
df_bp = df_bp[(df_bp['Unnamed: 0'] == 'Сальдо счета текущих операций и счета операций с капиталом') | (df_bp['Unnamed: 0'] == 'Сальдо финансового счета')]

In [51]:
df_bp = df_bp.T

In [52]:
df_bp = df_bp[1:]
df_bp.columns = ['Сальдо счета текущих операций и счета операций с капиталом', 'Сальдо финансового счета']

In [53]:
df_bp.index = range(df_bp.shape[0])

In [54]:
date_range = calendar_kvartal(datetime(1997, 9, 30), datetime(2023, 12, 31))

In [55]:
df_bp['Дата'] = date_range
df_bp = df_bp[['Дата','Сальдо счета текущих операций и счета операций с капиталом', 'Сальдо финансового счета']]
df_bp.head()

,Дата,Сальдо счета текущих операций и счета операций с капиталом,Сальдо финансового счета
0,1997-09-30,-1702.88,-3532.75
1,1997-12-30,-901.19,-4918.06
2,1998-03-30,-2962.14,-5440.26
3,1998-06-30,-3768.18,-6607.13
4,1998-09-30,787.81,-4091.7


In [56]:
df_bp['Дата'] = pd.to_datetime(df_bp['Дата'])
df_bp['Сальдо счета текущих операций и счета операций с капиталом'] = df_bp['Сальдо счета текущих операций и счета операций с капиталом'].astype(float)
df_bp['Сальдо финансового счета'] = df_bp['Сальдо финансового счета'].astype(float)

# Объединение в одну таблицу и дальнейшая работа 

создаю такой массив потмоу что 31 декабря биржа не работает и мы теряем значения макро показателей на 4 квартал в каждом году

In [68]:
date_range = calendar_po_dnyam(datetime(1997, 9, 22),datetime(2024, 1, 1))

In [69]:
df_merged = pd.DataFrame(date_range, columns=['Дата'])
df_merged['Дата'] = pd.to_datetime(df_merged['Дата'])
list_df = [df_bp, df_brent, df_gdp, df_ipc, df_key_rate, df_valut, df_imoex]

In [70]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9598 entries, 0 to 9597
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Дата    9598 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 75.1 KB


In [71]:
for i in list_df:
    df_merged = pd.merge(df_merged, i, on='Дата', how='left')

In [72]:
df_merged.head()

,Дата,Сальдо счета текущих операций и счета операций с капиталом,Сальдо финансового счета,"Цена, в $/барр","ВВП, в млрд. ₽","Инфляция, в %","Ключевая ставка, в %",Курс доллара к рублю,Цена закрытия
0,1997-09-22,NaN,NaN,18.63,NaN,NaN,NaN,5853.5,100.00
1,1997-09-23,NaN,NaN,18.64,NaN,NaN,NaN,5855.0,100.67
2,1997-09-24,NaN,NaN,18.80,NaN,NaN,NaN,5856.5,99.94
3,1997-09-25,NaN,NaN,19.14,NaN,NaN,NaN,5858.5,99.46
4,1997-09-26,NaN,NaN,19.56,NaN,NaN,NaN,5860.0,98.87


Мы вытягиваем значения за весь периолд а он в конце квартала 
потому используем bfill, также изьегаме того что еасли выкинем все nan'ы imoex то потеряем значения

In [74]:
df_merged[['Сальдо счета текущих операций и счета операций с капиталом', 'Сальдо финансового счета', 'ВВП, в млрд. ₽', 'Инфляция, в %', 'Ключевая ставка, в %']] = df_merged[['Сальдо счета текущих операций и счета операций с капиталом', 'Сальдо финансового счета', 'ВВП, в млрд. ₽', 'Инфляция, в %', 'Ключевая ставка, в %']].fillna(method='bfill')

In [77]:
df_merged[['Цена, в $/барр', 'Курс доллара к рублю']] = df_merged[['Цена, в $/барр', 'Курс доллара к рублю']].fillna(method='bfill')

Выбрасываем все на данное время пустые значения и оставляем только те даты на которые есть наблюдения по IMOEX

In [81]:
df_merged.dropna(inplace=True)

In [85]:
df_merged

,Дата,Сальдо счета текущих операций и счета операций с капиталом,Сальдо финансового счета,"Цена, в $/барр","ВВП, в млрд. ₽","Инфляция, в %","Ключевая ставка, в %",Курс доллара к рублю,Цена закрытия
0,1997-09-22,-1702.88,-3532.75,18.63,634.159200,99.70,21.0,5853.5000,100.00
1,1997-09-23,-1702.88,-3532.75,18.64,634.159200,99.70,21.0,5855.0000,100.67
2,1997-09-24,-1702.88,-3532.75,18.80,634.159200,99.70,21.0,5856.5000,99.94
3,1997-09-25,-1702.88,-3532.75,19.14,634.159200,99.70,21.0,5858.5000,99.46
4,1997-09-26,-1702.88,-3532.75,19.56,634.159200,99.70,21.0,5860.0000,98.87
...,...,...,...,...,...,...,...,...,...
9590,2023-12-25,10239.40,12961.50,81.07,50814.443887,100.73,16.0,91.9389,3099.02
9591,2023-12-26,10239.40,12961.50,81.07,50814.443887,100.73,16.0,91.9690,3094.72
9592,2023-12-27,10239.40,12961.50,79.54,50814.443887,100.73,16.0,91.7069,3097.49
9593,2023-12-28,10239.40,12961.50,77.15,50814.443887,100.73,16.0,91.7051,3101.99


In [95]:
df_merged[df_merged['Инфляция, в %'].pct_change() == df_merged['Инфляция, в %'].pct_change().min()]

,Дата,Сальдо счета текущих операций и счета операций с капиталом,Сальдо финансового счета,"Цена, в $/барр","ВВП, в млрд. ₽","Инфляция, в %","Ключевая ставка, в %",Курс доллара к рублю,Цена закрытия
374,1998-10-01,6093.89,6367.93,14.11,800.8474,104.54,55.0,15.9056,23.39
